From previous notebook to import and scrub the data from wikipedia

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#html_data=requests.get(url).text
#soup = BeautifulSoup(html_data.text, 'html.parser')
data = pd.read_html(url)

df_temp = data[0]

In [2]:
def str_breaker(str_in):
    pCode = str_in[0:3]
    if "Not assigned" in str_in:
        borough = "Not assigned"
        neighborhood = ''
    else:
        indy = str_in.index('(')
        borough = str_in[3:indy]
        #neighborhood = str_in[indy+1:-1].split(' / ')
        neighborhood = str_in[indy+1:].replace(' / ',', ')
        neighborhood = neighborhood.replace('(','')
        neighborhood = neighborhood.replace(')',' ')
    return pCode, borough, neighborhood

In [3]:
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
for i in range(df_temp.shape[0]):
    for j in range(df_temp.shape[1]):
        pC, bo, nh = str_breaker(df_temp.loc[i,j])
        if bo != "Not assigned":
            df = df.append({'PostalCode':pC, 'Borough':bo, 'Neighborhood':nh}, ignore_index = True)

In [4]:
df2 = pd.read_csv(r"Geospatial_Coordinates.csv")

In [5]:
df_final = df.join(df2.set_index('Postal Code'),on='PostalCode')

# New Code Begins Here
Import additional libraries (mostly copied from Manhattan Segmenting Lab)

In [6]:
import numpy as np
import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

Setup for Foursquare API (Credentials removed for publication)

In [7]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = ''

Following function was copied from lab as well (updated "Neighborhood" to "PostalCode" where need)

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
            
        # make the GET request
        #print(requests.get(url).json())
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create a data frame of boroughs that contain "Toronto" in their name

In [9]:
toronto_data = df_final[df_final['Borough'].astype(str).str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Following code gets the venues in a given post code

In [10]:
toronto_venues = getNearbyVenues(names=toronto_data['PostalCode'], latitudes=toronto_data['Latitude'], longitudes=toronto_data['Longitude'])

M5A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M4J
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


Size and head of the new dataframe

In [11]:
print(toronto_venues.shape)
toronto_venues.head()

(1592, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Venues per postal code

In [12]:
toronto_venues.groupby('PostalCode').count()

,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M4E,4,4,4,4,4,4
M4J,4,4,4,4,4,4
M4K,42,42,42,42,42,42
M4L,19,19,19,19,19,19
M4M,37,37,37,37,37,37
M4N,5,5,5,5,5,5
M4P,9,9,9,9,9,9
M4R,18,18,18,18,18,18
M4S,37,37,37,37,37,37


New data frame with the number of venue types in each column

In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move postal code column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group all rows with the same postal code and average by number of venues in that postal code

In [14]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

,PostalCode,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00000,0.000000,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4J,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,M4K,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,...,0.0000,0.02381,0.000000,0.02381,0.00,0.000000,0.000000,0.000000,0.000000,0.023810
3,M4L,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,M4M,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054054,0.000000,...,0.0000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.027027
5,M4N,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4P,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,M4R,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.055556
8,M4S,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00000,0.027027,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4T,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


The following function was derived from an earlier lab. It returns the 10 most common venues for an area

In [15]:
def return_10_most_common_venues(row):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:10]

Creating a new data frame with the 10 most common venues in that post code

Following code is also derived from previous lab. It creates a new data frame with the 10 most common venues

In [16]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(10):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pc_venues_sorted = pd.DataFrame(columns=columns)
pc_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    pc_venues_sorted.iloc[ind, 1:] = return_10_most_common_venues(toronto_grouped.iloc[ind, :])

pc_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Neighborhood,Trail,Pub,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
1,M4J,Park,Pizza Place,Convenience Store,Intersection,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
2,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Cosmetics Shop,Juice Bar
3,M4L,Sushi Restaurant,Pub,Brewery,Sandwich Place,Fast Food Restaurant,Restaurant,Fish & Chips Shop,Italian Restaurant,Intersection,Steakhouse
4,M4M,Coffee Shop,Brewery,Café,Gastropub,Bakery,American Restaurant,Coworking Space,Seafood Restaurant,Cheese Shop,Clothing Store


Run k-means clustering on the data frame with grouped data

In [17]:
# set number of clusters
kclusters = 12

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 7, 1, 6, 1, 4, 6, 1, 1, 8], dtype=int32)

Add cluster labels to the top 10 venues data frame

In [18]:
pc_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
pc_venues_sorted.head()

,Cluster Labels,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M4E,Health Food Store,Neighborhood,Trail,Pub,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
1,7,M4J,Park,Pizza Place,Convenience Store,Intersection,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
2,1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Cosmetics Shop,Juice Bar
3,6,M4L,Sushi Restaurant,Pub,Brewery,Sandwich Place,Fast Food Restaurant,Restaurant,Fish & Chips Shop,Italian Restaurant,Intersection,Steakhouse
4,1,M4M,Coffee Shop,Brewery,Café,Gastropub,Bakery,American Restaurant,Coworking Space,Seafood Restaurant,Cheese Shop,Clothing Store


Merge the previous data frame with borough and neighborhood data

In [19]:
toronto_final = toronto_data
toronto_final = toronto_final.join(pc_venues_sorted.set_index('PostalCode'), on='PostalCode')
toronto_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Cosmetics Shop,Shoe Store,Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Bubble Tea Shop,Cosmetics Shop,Café,Japanese Restaurant,Diner,Italian Restaurant,Bookstore,Ramen Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Cocktail Bar,Cosmetics Shop,Gastropub,Clothing Store,Restaurant,Bakery,Creperie,Department Store
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Neighborhood,Trail,Pub,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Cheese Shop,Park


Visualize data on a map. Code derived from lab.

In [20]:
geolocator = Nominatim(user_agent="ToTo")
location = geolocator.geocode("Toronto, ON, Canada")
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['PostalCode'], toronto_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [21]:
toronto_final.loc[toronto_final['Cluster Labels'] == 0, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M4E,0,Health Food Store,Neighborhood,Trail,Pub,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant


Cluster 2

In [22]:
toronto_final.loc[toronto_final['Cluster Labels'] == 1, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Cosmetics Shop,Shoe Store,Restaurant
1,M5B,1,Coffee Shop,Clothing Store,Bubble Tea Shop,Cosmetics Shop,Café,Japanese Restaurant,Diner,Italian Restaurant,Bookstore,Ramen Restaurant
2,M5C,1,Coffee Shop,Café,Cocktail Bar,Cosmetics Shop,Gastropub,Clothing Store,Restaurant,Bakery,Creperie,Department Store
4,M5E,1,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Cheese Shop,Park
5,M5G,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Salad Place,Restaurant,Portuguese Restaurant
7,M5H,1,Coffee Shop,Clothing Store,Café,Restaurant,Thai Restaurant,Bakery,Gym,Hotel,Steakhouse,Pizza Place
10,M5J,1,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Scenic Lookout,Italian Restaurant,Fried Chicken Joint,Brewery,History Museum
12,M4K,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Cosmetics Shop,Juice Bar
13,M5K,1,Coffee Shop,Café,Hotel,Restaurant,Salad Place,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Bakery,Sushi Restaurant
14,M6K,1,Café,Coffee Shop,Breakfast Spot,Convenience Store,Restaurant,Italian Restaurant,Stadium,Intersection,Bar,Climbing Gym


Cluster 3

In [23]:
toronto_final.loc[toronto_final['Cluster Labels'] == 2, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5N,2,Garden,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Cluster 4

In [24]:
toronto_final.loc[toronto_final['Cluster Labels'] == 3, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,M5P,3,Trail,Jewelry Store,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


Cluster 5

In [25]:
toronto_final.loc[toronto_final['Cluster Labels'] == 4, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M4N,4,Park,Lake,Bus Line,Dim Sum Restaurant,Swim School,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


Cluster 6

In [26]:
toronto_final.loc[toronto_final['Cluster Labels'] == 5, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,M4W,5,Park,Playground,Trail,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


Cluster 7

In [27]:
toronto_final.loc[toronto_final['Cluster Labels'] == 6, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,M4L,6,Sushi Restaurant,Pub,Brewery,Sandwich Place,Fast Food Restaurant,Restaurant,Fish & Chips Shop,Italian Restaurant,Intersection,Steakhouse
20,M4P,6,Gym,Hotel,Playground,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Gym / Fitness Center,General Entertainment
38,M7Y,6,Yoga Studio,Auto Workshop,Park,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop,Light Rail Station


Cluster 8

In [28]:
toronto_final.loc[toronto_final['Cluster Labels'] == 7, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M4J,7,Park,Pizza Place,Convenience Store,Intersection,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


Cluster 9

In [29]:
toronto_final.loc[toronto_final['Cluster Labels'] == 8, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,M4T,8,Park,Lawyer,Tennis Court,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


Cluster 10

In [30]:
toronto_final.loc[toronto_final['Cluster Labels'] == 9, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M6G,9,Grocery Store,Café,Park,Restaurant,Coffee Shop,Italian Restaurant,Candy Store,Nightclub,Baby Store,Dumpling Restaurant


Cluster 11

In [31]:
toronto_final.loc[toronto_final['Cluster Labels'] == 10, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M6H,10,Bakery,Pharmacy,Grocery Store,Music Venue,Bank,Supermarket,Middle Eastern Restaurant,Brewery,Café,Bus Stop
11,M6J,10,Bar,Restaurant,Vietnamese Restaurant,Asian Restaurant,Men's Store,Café,Yoga Studio,Beer Store,Bistro,Italian Restaurant
22,M6P,10,Mexican Restaurant,Thai Restaurant,Café,Grocery Store,Arts & Crafts Store,Speakeasy,Bar,Fried Chicken Joint,Bakery,Discount Store
25,M6R,10,Gift Shop,Breakfast Spot,Bookstore,Dessert Shop,Eastern European Restaurant,Bar,Dog Run,Italian Restaurant,Movie Theater,Restaurant
27,M5S,10,Café,Bookstore,Bar,Bakery,Sandwich Place,Japanese Restaurant,Beer Bar,Poutine Place,Beer Store,Bank
30,M5T,10,Bar,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Arts & Crafts Store,Burger Joint,Bakery,Park,Grocery Store


Cluster 12

In [32]:
toronto_final.loc[toronto_final['Cluster Labels'] == 11, toronto_final.columns[[0] + list(range(5, toronto_final.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M5V,11,Airport Service,Airport Lounge,Harbor / Marina,Coffee Shop,Boat or Ferry,Airport Terminal,Sculpture Garden,Airport Gate,Airport Food Court,Airport
